# How to use this notebook?

Firstly, you need to install the dependencies on your Python environment.

```
pip install -r requirements.txt
```

Then you need to execute `wikimedia-api.py` script to generate `queried_datasets.csv` file.

And at the end you can run this notebook to clean the dataset...

In [8]:
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
df = pd.read_csv('queried_datasets.csv')

In [6]:
df.drop('Unnamed: 0',inplace=True,axis=1)
df.head()

,id,key,title,excerpt,matched_title,description,thumbnail
0,22261,Ontology,Ontology,"metaphysics, <span class=""searchmatch"">ontolog...",NaN,Branch of philosophy that studies concepts suc...,"{'mimetype': 'image/png', 'size': None, 'width..."
1,2477036,Guerrilla_ontology,Guerrilla ontology,"Guerilla <span class=""searchmatch"">ontology</s...",NaN,NaN,NaN
2,49681,Ontology_(information_science),Ontology (information science),"In computer science and information science, a...",NaN,Specification of a conceptualization,NaN
3,1454791,Gene_Ontology,Gene Ontology,"The Gene <span class=""searchmatch"">Ontology</s...",NaN,Bioinformatics initiative,NaN
4,3200382,Upper_ontology,Upper ontology,"information science, an upper <span class=""sea...",NaN,Ontology applicable across domains of knowledge,NaN


In [10]:
df['excerpt'] = [BeautifulSoup(X).getText() for X in df['excerpt']]

In [11]:
df.head()

,id,key,title,excerpt,matched_title,description,thumbnail
0,22261,Ontology,Ontology,"metaphysics, ontology is the philosophical stu...",NaN,Branch of philosophy that studies concepts suc...,"{'mimetype': 'image/png', 'size': None, 'width..."
1,2477036,Guerrilla_ontology,Guerrilla ontology,Guerilla ontology is a practice described by a...,NaN,NaN,NaN
2,49681,Ontology_(information_science),Ontology (information science),"In computer science and information science, a...",NaN,Specification of a conceptualization,NaN
3,1454791,Gene_Ontology,Gene Ontology,The Gene Ontology (GO) is a major bioinformati...,NaN,Bioinformatics initiative,NaN
4,3200382,Upper_ontology,Upper ontology,"information science, an upper ontology (also k...",NaN,Ontology applicable across domains of knowledge,NaN


In [12]:
df.to_csv('queried_datasets_excerpt_cleaned.csv')